In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Third-party imports
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import random
import numpy as np


from index_travel_accessibility.travel_time_and_centroid import (
    map_predicted_and_existing_hospitals,
)

# Local imports
from ai_planner.planner import (
    HospitalPlanner,
    CRS
)

from index_travel_accessibility.travel_time_and_centroid import (
    get_hospital_df,
)
from main import (
    Index,
    assemble_indexes,
    centroid,
    current_hospital_demand,
    equity_index,
)

In [2]:
# Load sheet with correct name and skip header rows
df = pd.read_excel("./data/kreise.xlsx", sheet_name="Kreisfreie Städte u. Landkreise", skiprows=6)

# Drop rows without valid district codes (will be NaN for header/total rows)
df = df[df[df.columns[0]].notna()]

# Keep only Saarland districts (codes start with '10' and are 5-digit numbers)
df = df[df[df.columns[0]].astype(str).str.startswith("10") & (df[df.columns[0]].astype(str).str.len() == 5)]

# Extract district name and population
result = df[[df.columns[0], df.columns[2], df.columns[5]]].copy()
result.columns = ["District_code","District", "Population"]

print(result)


    District_code                     District  Population
374         10041  Regionalverband Saarbrücken    332427.0
375         10042                Merzig-Wadern    104327.0
376         10043                  Neunkirchen    132393.0
377         10044                    Saarlouis    195945.0
378         10045              Saarpfalz-Kreis    142344.0
379         10046                   St. Wendel     86988.0


In [3]:
total_k = 10
b_n = 2  # number of least populated districts to exclude

df = result.copy()

df = df.sort_values("Population", ascending=False).reset_index(drop=True)
df["Hospitals"] = 0  # default

top_df = df.iloc[:-b_n].copy()
total_pop = top_df["Population"].sum()

top_df["Hospitals"] = (top_df["Population"] / total_pop * total_k).round().astype(int)

df.update(top_df)

while df["Hospitals"].sum() != total_k:
    diff = total_k - df["Hospitals"].sum()
    idx = df["Hospitals"].idxmax() if diff < 0 else df["Hospitals"].idxmin()
    df.at[idx, "Hospitals"] += 1 if diff > 0 else -1


In [ ]:
# def policy_maker_model(df):
#     """
#     Population-weighted hospital allocation with realistic coordinates inside district geometries.
#     """
#     # Total max beds across all hospitals
#     TOTAL_BEDS = 1500

#     # Filter districts with hospital assignments
#     eligible_df = df[df["Hospitals"] > 0].reset_index(drop=True)

#     # Compute proportional max bed per district
#     eligible_df["BedQuota"] = (eligible_df["Population"] / eligible_df["Population"].sum()) * TOTAL_BEDS

#     # Generate hospital data
#     hospital_list = []
#     node_counter = 1

#     for _, row in eligible_df.iterrows():
#         district = row["District"]
#         pop = row["Population"]
#         num_hosp = row["Hospitals"]
#         bed_quota = row["BedQuota"]
#         district_code = row["District_code"]

#         for _ in range(num_hosp):
#             # Generate random coordinates within Saarland bounding box
#             lat = round(random.uniform(49.1, 49.6), 6)
#             lon = round(random.uniform(6.5, 7.5), 6)

#             # Approximate bed allocation
#             bed = max(50, int(np.random.normal(loc=bed_quota/num_hosp, scale=20)))

#             hospital_list.append({
#                 "geometry": Point(lon, lat),
#                 "district_code": district_code,
#                 "node": f'k{node_counter}',
#                 "bed_allocation": bed,
#                 "Lat": lat,
#                 "Lon": lon,
#                 "type": "prediction"
#             })
#             node_counter += 1

#     # Create DataFrame
#     hospital_df = pd.DataFrame(hospital_list)

#     # Normalize beds to total 1500 exactly
#     scaling_factor = TOTAL_BEDS / hospital_df["bed_allocation"].sum()
#     hospital_df["bed_allocation"] = (hospital_df["bed_allocation"] * scaling_factor).round().astype(int)

#     # Final fix to match total exactly
#     while hospital_df["bed_allocation"].sum() != TOTAL_BEDS:
#         diff = TOTAL_BEDS - hospital_df["bed_allocation"].sum()
#         idx = hospital_df["bed_allocation"].idxmax() if diff < 0 else hospital_df["bed_allocation"].idxmin()
#         hospital_df.at[idx, "bed_allocation"] += 1 if diff > 0 else -1


#     return hospital_df

In [7]:
def random_point_in_polygon(polygon):
    minx, miny, maxx, maxy = polygon.bounds
    while True:
        p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(p):
            return p

def policy_maker_model(df, nuts):
    TOTAL_BEDS = 1500

    # Map district codes to official district names (ensure matching with nuts)
    code_to_name = {
        "10041": "Regionalverband Saarbrücken",
        "10042": "Merzig-Wadern",
        "10043": "Neunkirchen",
        "10044": "Saarlouis",
        "10045": "Saarpfalz-Kreis",
        "10046": "St. Wendel"
    }

    # Filter districts with hospitals > 0
    eligible_df = df[df["Hospitals"] > 0].copy()
    eligible_df["district_name"] = eligible_df["District_code"].map(code_to_name)

    # Merge with nuts polygons to get geometry for each district
    merged = eligible_df.merge(nuts[['NUTS_NAME', 'geometry']], left_on='district_name', right_on='NUTS_NAME', how='left')

    if merged['geometry'].isnull().any():
        missing = merged[merged['geometry'].isnull()]
        raise ValueError(f"Missing geometry for districts: {missing['district_name'].tolist()}")

    # Compute bed quota per district based on population
    merged["BedQuota"] = (merged["Population"] / merged["Population"].sum()) * TOTAL_BEDS

    hospital_list = []
    node_counter = 1

    for _, row in merged.iterrows():
        polygon = row["geometry"]
        num_hosp = int(row["Hospitals"])
        bed_quota = row["BedQuota"]
        district_code = row["District_code"]

        for _ in range(num_hosp):
            point = random_point_in_polygon(polygon)
            bed = max(50, int(np.random.normal(loc=bed_quota / num_hosp, scale=20)))

            hospital_list.append({
                "geometry": point,
                "district_code": district_code,
                "node": f'k{node_counter}',
                "bed_allocation": bed,
                "Lat": point.y,
                "Lon": point.x,
                "type": "predicted"
            })
            node_counter += 1

    hospital_df = gpd.GeoDataFrame(hospital_list, geometry='geometry', crs='EPSG:4326')

    # Normalize beds so sum is exactly TOTAL_BEDS
    scaling_factor = TOTAL_BEDS / hospital_df["bed_allocation"].sum()
    hospital_df["bed_allocation"] = (hospital_df["bed_allocation"] * scaling_factor).round().astype(int)

    while hospital_df["bed_allocation"].sum() != TOTAL_BEDS:
        diff = TOTAL_BEDS - hospital_df["bed_allocation"].sum()
        idx = hospital_df["bed_allocation"].idxmax() if diff < 0 else hospital_df["bed_allocation"].idxmin()
        hospital_df.at[idx, "bed_allocation"] += 1 if diff > 0 else -1

    return hospital_df

nuts = gpd.read_file('./../index_travel_accessibility/data/raw/NUTS_RG_01M_2021_4326_LEVL_3.shp')
nuts = nuts.to_crs('EPSG:4326')




In [8]:
selected_candidates = policy_maker_model(df, nuts)

In [9]:
selected_candidates

,geometry,district_code,node,bed_allocation,Lat,Lon,type
0,POINT (6.88339 49.32609),10041,k1,129,49.326095,6.883387,predicted
1,POINT (6.92947 49.26813),10041,k2,182,49.268129,6.929471,predicted
2,POINT (7.00528 49.33768),10041,k3,150,49.337678,7.005279,predicted
3,POINT (6.96904 49.31349),10041,k4,177,49.313489,6.969039,predicted
4,POINT (6.82262 49.35667),10044,k5,190,49.356668,6.822621,predicted
5,POINT (6.90422 49.44209),10044,k6,177,49.442089,6.904225,predicted
6,POINT (7.2548 49.35111),10045,k7,126,49.351108,7.254802,predicted
7,POINT (7.28003 49.3762),10045,k8,127,49.376205,7.280035,predicted
8,POINT (7.17252 49.40974),10043,k9,108,49.409736,7.172517,predicted
9,POINT (7.21336 49.39719),10043,k10,134,49.397190,7.213364,predicted


In [11]:
map_predicted_and_existing_hospitals(
    'results/maps/policy_maker_model.html', 
    selected_candidates
)

✅ Map saved with existing and predicted hospitals!


In [12]:
# Modifying the existing hospitals DataFrame to Prediction DataFrame format
existing_hospitals = get_hospital_df()

existing_hospitals.rename(columns={
    'MaxBeds': 'bed_allocation',
    'SiteID': 'node'
}, inplace=True)


# Adding Geometry column to existing_hospitals
existing_hospitals['Lat'] = existing_hospitals['Lat'].round(6)
existing_hospitals['Lon'] = existing_hospitals['Lon'].round(6)
existing_hospitals['geometry'] = existing_hospitals.apply(lambda row: Point(row['Lon'], row['Lat']), axis=1)

# Getting district codes from the existing hospitals

# Load Saarland NUTS level 3 shapefile
nuts = gpd.read_file('./../index_travel_accessibility/data/raw/NUTS_RG_01M_2021_4326_LEVL_3.shp')
nuts = nuts.to_crs('EPSG:4326')


# Create geometry column from Lat/Lon
# existing_hospitals['geometry'] = existing_hospitals.apply(lambda row: Point(row['Lon'], row['Lat']), axis=1)
# existing_hospitals = gpd.GeoDataFrame(existing_hospitals, geometry='geometry', crs='EPSG:4326')

existing_hospitals = gpd.GeoDataFrame(existing_hospitals, geometry='geometry', crs='EPSG:4326')

# Spatial join hospitals with NUTS polygons (only needed columns)
existing_hospitals = gpd.sjoin(
    existing_hospitals,
    nuts[['geometry', 'NUTS_NAME']],
    how='left',
    predicate='within'
)

# Map NUTS_NAME to your official Saarland district codes
name_to_code = {
    "Regionalverband Saarbrücken": "10041",
    "Merzig-Wadern": "10042",
    "Neunkirchen": "10043",
    "Saarlouis": "10044",
    "Saarpfalz-Kreis": "10045",
    "St. Wendel": "10046"
}

# Assign district_code based on district_name
existing_hospitals['district_code'] = existing_hospitals['NUTS_NAME'].map(name_to_code)

# Rename NUTS_NAME to district_name
existing_hospitals.rename(columns={'NUTS_NAME': 'district_name'}, inplace=True)

# Select and reorder columns as needed
existing_hospitals = existing_hospitals[['geometry', 'district_code', 'node', 'bed_allocation', 'Lat', 'Lon']]


existing_hospitals.head()



,geometry,district_code,node,bed_allocation,Lat,Lon
0,POINT (7.04132 49.27707),10041,H1,157.0,49.277066,7.041318
1,POINT (6.95997 49.24756),10041,H2,441.0,49.247555,6.959970
2,POINT (6.99259 49.23708),10041,H3,2.0,49.237085,6.992590
3,POINT (6.99414 49.22122),10041,H4,571.0,49.221221,6.994143
4,POINT (6.96355 49.247),10041,H5,277.0,49.246999,6.963554


In [13]:
existing_hospitals['type'] = 'existing'

final_results = pd.concat([existing_hospitals, selected_candidates], ignore_index=True)
final_results.to_excel("results/policy_maker_model.xlsx", index=False)